# Import necesarry modules 

In [1]:
%pylab notebook

Populating the interactive namespace from numpy and matplotlib


In [2]:
from Fisher_CMB import *
from Models.SPT3G.spt3G_model import spt3G_model
from Models.Planck.Planck_model import planck_model
import pickle as p
from Covariances.create_basic_cov import *
import pandas as pd
import cosmoslik as K
import matplotlib.lines as mlines

# Define needed functions

In [3]:
def SPT3G_Fisher(params,fsky,saveas,
                 beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
                 noise=[array([3.6,3.3,8.5]),'ukarc'],
                 lmin=50,
                 P_lmax=5000,
                 T_lmax=3000,
                 spectra=['TT','EE','TE']):
    '''Calculate a SPT3G bandpower covariance, fisher matrix, and save as pandas dataframe
    '''

    spt3g_cov=create_basic_cov(fsky,beam_width,noise,specrange = (lmin,max([T_lmax,P_lmax])),
                               bin_size = 5,spectra=spectra)
    full_spt3g_cov=make_full_cov(spectra,spt3g_cov)
    Fs,names=Fisher_Matrix(params,full_spt3g_cov,
                spt3G_model(specrange = list(zip(spectra,[(lmin,T_lmax),(lmin,P_lmax),(lmin,P_lmax)])),
                             bin_size=5),show_results=False)
    pd.DataFrame(Fs,index=names,columns=names).to_hdf(saveas,'key')

In [14]:
def SPT3G_phi_Fisher(params,fsky,saveas,
                     lmin=50,lmax=3000):
    '''Calculate a SPT3G phiphi fisher matrix from previously made covariance
    '''
    m=spt3G_model(specrange = [('PP',(lmin,lmax))],bin_size=5)
    lensing_recon_cov_file = 'Covariances/spt3g_3.3uk_lmaxTT3000_lmaxPOL5000_nx1024_plikHM_TT_lowTEB_lensing.dat'
    PP_cov=diag(dot(m.windows['PP'],
                (loadtxt(lensing_recon_cov_file)[:,-1]**2*(2.725*1e6)**4)[m.windowrange['PP']]))
    Fs_phi,names=Fisher_Matrix(params,PP_cov,
                                spt3G_model(specrange = [('PP',(lmin,lmax))],bin_size=5),show_results=False)
    pd.DataFrame(Fs_phi,index=names,columns=names).to_hdf(saveas,'key')

In [5]:
def Planck_800_Fisher(params,saveas):
    '''Calculate a planck lmin<800 bandpower covariance, fisher matrix, and save as pandas dataframe
    '''
    pcov_800=create_basic_cov(.7,[.0003],[0,'ukarc'],specrange = (2,800),bin_size = 5,spectra=['TT'])
    full_pcov_800 = make_full_cov(['TT'],pcov_800)
    Fp_800,names=Fisher_Matrix(params,full_pcov_800,
                 planck_model(specrange = [('TT',(2,800))],bin_size=5),show_results=False)
    pd.DataFrame(Fp_800,index=names,columns=names).to_hdf(saveas,'key')

In [23]:
def Planck_chain_Fisher(params,chain_path,saveas):
    '''Calculate a Planck Fisher matrix from a chain'''
    
    Planck_chain=K.chains.load_cosmomc_chain(chain_path).burnin(1000).join()
    Planck_chain=add_params(Planck_chain)
    cov_params=sorted(params)
    Planck_cov = Planck_chain.cov(cov_params)
    Fp=inv(Planck_cov)
    pd.DataFrame(Fp,index=cov_params,columns=cov_params).to_hdf(saveas,'key')

In [7]:
def SPTSZ_chain_Fisher(params,chain_path,saveas):
    '''Calculate a SPTSZ Fisher matrix from a chain'''
    
    SPTSZ_chain=K.load_chain(chain_path).burnin(500).join()
    SPTSZ_chain = slik2cmc(SPTSZ_chain)
    cov_params=sorted(params)
    SPTSZ_cov = SPTSZ_chain.cov(sorted(cov_params))
    Fsz=inv(SPTSZ_cov)
    pd.DataFrame(Fsz,index=cov_params,columns=cov_params).to_hdf(saveas,'key')

In [25]:
def add_params(planck):
    '''Add parameters to planck chain'''
    planck['ommh2']=planck['omegabh2']+planck['omegach2']
    planck['H0']=planck['H0*']
    planck['clamp']=planck['clamp*']
    planck['ombh2']=planck['omegabh2']
    planck['YHe']=planck.get('yhe',planck['H0*'])
    planck['omk']=planck.get('omegak',planck['H0*'])
    return planck

In [9]:
def slik2cmc( c ):
    '''Convert slik params to names in use'''
    c['ns'] = c['cosmo.ns']
    c['ombh2'] = c['cosmo.ombh2']
    c['omch2'] = c['cosmo.omch2']
    c['tau'] = c['cosmo.tau']
    c['clamp'] = exp(c['cosmo.logA'])*exp(-2*c['tau'])/10
    c['ommh2']=c['ombh2']+c['omch2']
    c['H0'] = c['cosmo.H0']
    return c

# Make Fisher Matrices and Save 

##  LCDM Params

In [10]:
params_base={'H0':fish_param(67.90,0.55),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [11]:
params3g_base={'H0':fish_param(67.90,0.55),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                            'Asz':fish_param(5.5,3,3,nuisance=True),
                            'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            'A_TEps' : fish_param(.16,.05,nuisance=True),
                            'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

### Extensions

In [9]:
model_extensions=[('lcdm_',{}),
                  ('lcdm_neff',{'nnu':fish_param(3.046,0.03)}),
                  ('lcdm_neff_yp',{'nnu':fish_param(3.046,0.03),
                           'YHe':fish_param(.252,.014),}),
                  ('lcdm_mnu',{'mnu':fish_param(.06,0.01)}),
                  ('lcdm_omk',{'omk':fish_param(-0.003687,0.007)})]

### Planck TT $\ell$ < 800 Fisher matrix

In [13]:
for k in model_extensions:
    params=dict(params_base,**k[1])
    Planck_800_Fisher(params,'Saved_Fisher_Matrices/planck_TT_'+k[0]+'_800_Fisher_Matrix.h5')

{'lens_potential_accuracy': 2.0, 'As': 2.139e-09, 'ombh2': 0.02227, 'omch2': 0.1184, 'H0': None, 'tau': 0.067, 'ns': 0.9681, 'cosmomc_theta': 0.0104106, 'lmax': 6000}


/nfs/home/kmaylor/Forecasting/Covariances/create_basic_cov.py:37: RuntimeWarning: divide by zero encountered in true_divide
  con=lambda n:arcmin_per_sr/n**2


{'H0': 68.17500000000001, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.62500000000001, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1481492599073809e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1350002424814674e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9702999999999999, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 

{'H0': 68.17500000000001, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.62500000000001, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1481492599073809e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1350002424814674e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.065, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.055, 'ombh2': 0.02227, 'omch2':

### SPT3G Fisher Matrix

In [15]:
for k in model_extensions:
    params=dict(params3g_base,**k[1])
    SPT3G_Fisher(params,0.06,'Saved_Fisher_Matrices/SPT3G_'+k[0]+'_lmin50_Fisher_Matrices.h5')

{'lens_potential_accuracy': 2.0, 'As': 2.139e-09, 'ombh2': 0.02227, 'omch2': 0.1184, 'H0': None, 'tau': 0.067, 'ns': 0.9681, 'cosmomc_theta': 0.0104106, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 6

{'H0': 67.9, 'As': 2.1139144608640354e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.06050000000000001, 'ns': 0.9681, 'lmax': 6000}
{'lens_potential_accuracy': 2.0, 'As': 2.139e-09, 'ombh2': 0.02227, 'omch2': 0.1184, 'H0': None, 'tau': 0.067, 'ns': 0.9681, 'cosmomc_theta': 0.0104106, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000, 'YHe': 0.252}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000, 'YHe': 0.252}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000, 'YHe': 0.252}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 

{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.055, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9702999999999999, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9659, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02237, 'omch2': 0.11893000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.022170000000000002, 'omch2': 0.11913000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0

### SPT3G Fisher Matrix fsky =0.035

In [16]:
for k in model_extensions:
    params=dict(params3g_base,**k[1])
    SPT3G_Fisher(params,0.035,'Saved_Fisher_Matrices/SPT3G_f035_'+k[0]+'_lmin50_Fisher_Matrices.h5')

{'lens_potential_accuracy': 2.0, 'As': 2.139e-09, 'ombh2': 0.02227, 'omch2': 0.1184, 'H0': None, 'tau': 0.067, 'ns': 0.9681, 'cosmomc_theta': 0.0104106, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 6

{'H0': 67.9, 'As': 2.1139144608640354e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.06050000000000001, 'ns': 0.9681, 'lmax': 6000}
{'lens_potential_accuracy': 2.0, 'As': 2.139e-09, 'ombh2': 0.02227, 'omch2': 0.1184, 'H0': None, 'tau': 0.067, 'ns': 0.9681, 'cosmomc_theta': 0.0104106, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000, 'YHe': 0.252}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000, 'YHe': 0.252}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 'nnu': 3.046, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000, 'YHe': 0.252}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'ombh2': 0.02227, 

{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.055, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9702999999999999, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0.11903000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9659, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02237, 'omch2': 0.11893000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.022170000000000002, 'omch2': 0.11913000000000001, 'lens_potential_accuracy': 5.0, 'tau': 0.067, 'ns': 0.9681, 'lmax': 6000}
{'H0': 67.9, 'As': 2.1415747511944239e-09, 'mnu': 0.06, 'ombh2': 0.02227, 'omch2': 0

###  SPT3G lensing reconstruction Fisher Matrix

In [15]:
for k in model_extensions:
    params=dict(params_base,**k[1])
    SPT3G_phi_Fisher(params,0.06,'Saved_Fisher_Matrices/SPT3G_phi_'+k[0]+'_lmin50_Fisher_Matrices.h5')

In [16]:
for k in model_extensions:
    params=dict(params_base,**k[1])
    SPT3G_phi_Fisher(params,0.035,'Saved_Fisher_Matrices/SPT3G_phi_f035_'+k[0]+'_lmin50_Fisher_Matrices.h5')

### plikHM_TTTEEE_lowTEB Fisher Matrix

In [95]:
planck_paths=['base/plikHM_TTTEEE_lowTEB/base_plikHM_TTTEEE_lowTEB',
             'base_nnu/plikHM_TTTEEE_lowTEB/base_nnu_plikHM_TTTEEE_lowTEB',
             'base_nnu_yhe/plikHM_TTTEEE_lowTEB/base_nnu_yhe_plikHM_TTTEEE_lowTEB',
             'base_mnu/plikHM_TTTEEE_lowTEB/base_mnu_plikHM_TTTEEE_lowTEB',
             'base_omegak/plikHM_TTTEEE_lowTEB/base_omegak_plikHM_TTTEEE_lowTEB']

In [87]:
for i,k in enumerate(model_extensions):
    chain_path='/nfs/home/kmaylor/Official_Planck_chains/'+planck_paths[i]
    params=dict(params_base,**k[1])
    chain_params=[k for k,v in params.items() if isinstance(v,fish_param)]
    Planck_chain_Fisher(chain_params,chain_path,
                        'Saved_Fisher_Matrices/planck_TTTEEE_lowTEB'+k[0]+'_Fisher_Matrix.h5')

### plikHM_TTTEEE_lowTEB_lensing Fisher Matrix

In [21]:
planck_paths=['base/plikHM_TTTEEE_lowTEB_lensing/base_plikHM_TTTEEE_lowTEB_lensing',
             'base_nnu/plikHM_TTTEEE_lowTEB_nnu1_lensing/base_nnu_plikHM_TTTEEE_lowTEB_nnu1_lensing',
             'base_nnu_yhe/plikHM_TTTEEE_lowTEB_lensing/base_nnu_yhe_plikHM_TTTEEE_lowTEB_lensing',
             'base_mnu/plikHM_TTTEEE_lowTEB_lensing/base_mnu_plikHM_TTTEEE_lowTEB_lensing',
             'base_omegak/plikHM_TTTEEE_lowTEB_lensing/base_omegak_plikHM_TTTEEE_lowTEB_lensing']

In [26]:
for i,k in enumerate(model_extensions):
    if i in [1,2]:continue
    chain_path='/nfs/home/kmaylor/Official_Planck_chains/'+planck_paths[i]
    params=dict(params_base,**k[1])
    chain_params=[k for k,v in params.items() if isinstance(v,fish_param)]
    Planck_chain_Fisher(chain_params,chain_path,
                        'Saved_Fisher_Matrices/planck_TTTEEE_lowTEB_lensing'+k[0]+'_Fisher_Matrix.h5')

### plikHM_TTTEEE_lowTEB_lensing_BAO Fisher Matrix

In [109]:
planck_paths=['base/plikHM_TTTEEE_lowTEB_lensing_BAO/base_plikHM_TTTEEE_lowTEB_lensing_BAO',
             'base_nnu/plikHM_TTTEEE_lowTEB_nnu1_lensing_BAO/base_nnu_plikHM_TTTEEE_lowTEB_nnu1_lensing_BAO',
             'base_nnu_yhe/plikHM_TTTEEE_lowTEB_lensing_BAO/base_nnu_yhe_plikHM_TTTEEE_lowTEB_lensing_BAO',
             'base_mnu/plikHM_TTTEEE_lowTEB_lensing_BAO/base_mnu_plikHM_TTTEEE_lowTEB_lensing_BAO',
             'base_omegak/plikHM_TTTEEE_lowTEB_lensing_BAO/base_omegak_plikHM_TTTEEE_lowTEB_lensing_BAO']

In [111]:
for i,k in enumerate(model_extensions):
    if i in [0,1,2,4]:continue
    chain_path='/nfs/home/kmaylor/Official_Planck_chains/'+planck_paths[i]
    params=dict(params_base,**k[1])
    chain_params=[k for k,v in params.items() if isinstance(v,fish_param)]
    Planck_chain_Fisher(chain_params,chain_path,
                        'Saved_Fisher_Matrices/planck_TTTEEE_lowTEB_lensing_BAO'+k[0]+'_Fisher_Matrix.h5')

In [30]:
p1=K.chains.load_cosmomc_chain('/nfs/home/kmaylor/Official_Planck_chains/base_mnu/plikHM_TTTEEE_lowTEB/base_mnu_plikHM_TTTEEE_lowTEB').burnin(1000).join()

In [31]:
p2=K.chains.load_cosmomc_chain('/nfs/home/kmaylor/Official_Planck_chains/base_mnu/plikHM_TTTEEE_lowTEB_lensing/base_mnu_plikHM_TTTEEE_lowTEB_lensing').burnin(1000).join()

### SPT-SZ Fisher Matrix

In [118]:
chain_params=[k for k,v in params_base.items() if isinstance(v,fish_param)]

In [119]:
chain_path="/home/kmaylor/Python_Projects/cosmology_chains/SPT_chainz_150x150.chains"
SPTSZ_chain_Fisher(chain_params,chain_path,'Saved_Fisher_Matrices/SPTSZ_lcdm_Fisher_Matrix.h5')

# Load and Combine Planck and SPT FIsher Matrices

In [27]:
F={}
for i,k in enumerate(model_extensions):
    f={}
    #SPT3G
    f['SPT3G_fsky_0.035']=pd.read_hdf(
        'Saved_Fisher_Matrices/SPT3G_f035_'+k[0]+'_lmin50_Fisher_Matrices.h5', 'key')
    f['SPT3G_fsky_0.06']=pd.read_hdf(
        'Saved_Fisher_Matrices/SPT3G_'+k[0]+'_lmin50_Fisher_Matrices.h5', 'key')
    #SPT3G_clphiphi
    f['SPT3G_clphiphi_fsky_0.06']=pd.read_hdf(
        'Saved_Fisher_Matrices/SPT3G_phi_'+k[0]+'_lmin50_Fisher_Matrices.h5', 'key')
    f['SPT3G_clphiphi_fsky_0.035']=pd.read_hdf(
        'Saved_Fisher_Matrices/SPT3G_phi_f035_'+k[0]+'_lmin50_Fisher_Matrices.h5', 'key')
    #SPT3G_lensing
    f['SPT3G_lensing_fsky_0.035']=f['SPT3G_fsky_0.035'].add(f['SPT3G_clphiphi_fsky_0.035'])
    f['SPT3G_lensing_fsky_0.06']=f['SPT3G_fsky_0.06'].add(f['SPT3G_clphiphi_fsky_0.06'])
    #Planck_lmax800
    f['Planck_TT_lmax_800']=pd.read_hdf(
        'Saved_Fisher_Matrices/planck_TT_'+k[0]+'_800_Fisher_Matrix.h5', 'key')
    #Planck_lmax800 and SPT3G
    f['Planck_TT_lmax_800_SPT3G_fsky_0.035']=f['Planck_TT_lmax_800'].add(f['SPT3G_fsky_0.035'])
    f['Planck_TT_lmax_800_SPT3G_fsky_0.06']=f['Planck_TT_lmax_800'].add(f['SPT3G_fsky_0.06'])
    f['Planck_TT_lmax_800_SPT3G_lensing_fsky_0.035']=f['Planck_TT_lmax_800_SPT3G_fsky_0.035'].add(
        f['SPT3G_clphiphi_fsky_0.035'])
    f['Planck_TT_lmax_800_SPT3G_lensing_fsky_0.06']=f['Planck_TT_lmax_800_SPT3G_fsky_0.06'].add(
        f['SPT3G_clphiphi_fsky_0.06'])
    #Planck_TTTEEE_lowTEB
    f['Planck_TTTEEE_lowTEB']=pd.read_hdf(
        'Saved_Fisher_Matrices/planck_TTTEEE_lowTEB'+k[0]+'_Fisher_Matrix.h5', 'key')
    #Planck_TTTEEE_lowTEB and SPT3G
    f['Planck_TTTEEE_lowTEB_SPT3G_fsky_0.035']=f['Planck_TTTEEE_lowTEB'].add(f['SPT3G_fsky_0.035'])
    f['Planck_TTTEEE_lowTEB_SPT3G_fsky_0.06']=f['Planck_TTTEEE_lowTEB'].add(f['SPT3G_fsky_0.06'])
    f['Planck_TTTEEE_lowTEB_SPT3G_lensing_fsky_0.035']=f['Planck_TTTEEE_lowTEB'].add(
        f['SPT3G_lensing_fsky_0.035'])
    f['Planck_TTTEEE_lowTEB_SPT3G_lensing_fsky_0.06']=f['Planck_TTTEEE_lowTEB'].add(
        f['SPT3G_lensing_fsky_0.06'])
    #Planck_TTTEEE_lowTEB_lensing and SPT3G_lensing
    if i not in [1,2]:
        f['Planck_TTTEEE_lowTEB_lensing']=pd.read_hdf(
            'Saved_Fisher_Matrices/planck_TTTEEE_lowTEB_lensing'+k[0]+'_Fisher_Matrix.h5', 'key')
        f['Planck_TTTEEE_lowTEB_lensing_SPT3G_lensing_fsky_0.06']=f['Planck_TTTEEE_lowTEB_lensing'].add(
            f['SPT3G_lensing_fsky_0.06'])
        f['Planck_TTTEEE_lowTEB_lensing_SPT3G_lensing_fsky_0.035']=f['Planck_TTTEEE_lowTEB_lensing'].add(
            f['SPT3G_lensing_fsky_0.035'])
        #Planck_TTTEEE_lowTEB_lensing_BAO and SPT3G_lensing
    if i not in [0,1,2,4]:
        f['Planck_TTTEEE_lowTEB_lensing_BAO']=pd.read_hdf(
            'Saved_Fisher_Matrices/planck_TTTEEE_lowTEB_lensing_BAO'+k[0]+'_Fisher_Matrix.h5', 'key')
        f['Planck_TTTEEE_lowTEB_lensing_BAO_SPT3G_lensing_fsky_0.06']=f['Planck_TTTEEE_lowTEB_lensing'].add(
            f['SPT3G_lensing_fsky_0.06'])
        f['Planck_TTTEEE_lowTEB_lensing_BAO_SPT3G_lensing_fsky_0.035']=f['Planck_TTTEEE_lowTEB_lensing'].add(
            f['SPT3G_lensing_fsky_0.035'])
        #SPTSZ
    if i ==0:
        f['SPTSZ']=pd.read_hdf('Saved_Fisher_Matrices/SPTSZ_lcdm_Fisher_Matrix.h5', 'key')

    F[k[0]]=pd.concat(f).transpose()

In [28]:
p.dump(F,open('All_Planck_and_SPT3G_Fisher_matrices.p','wb'))

In [30]:

G=p.load(open('All_Planck_and_SPT3G_Fisher_matrices.p','rb'))

{'lcdm_':       Planck_TTTEEE_lowTEB                                             \
                         H0         clamp             ns         ombh2   
 H0              109.368512    -41.069528    -429.001591 -1.482736e+05   
 clamp           -41.069528  10066.661288    3078.416168 -3.506980e+04   
 ns             -429.001591   3078.416168  111438.931807  2.301830e+05   
 ombh2       -148273.583059 -35069.795800  230182.989608  2.628671e+08   
 ommh2         39489.308448 -58403.443942   81991.264559 -5.055594e+07   
 tau               8.619230    293.824008   -4982.370283 -8.684540e+04   
 
                                   Planck_TTTEEE_lowTEB_SPT3G_fsky_0.035  \
               ommh2           tau                                    H0   
 H0     3.948931e+04      8.619230                            275.994506   
 clamp -5.840344e+04    293.824008                           -137.125523   
 ns     8.199126e+04  -4982.370283                           -817.698624   
 ombh2 -5.055594e